# **Chicago Crime Data vs Weather - 2016** 

In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('Crime_Weather_2016.csv')
crime_categories = pd.read_csv('CrimeCategories.csv')

#### Here we convert the data back into datetime format

In [7]:
df['datetime'] = pd.to_datetime(df['datetime'],format='%d/%m/%Y %H:%M')
print(df['datetime'].dtype)

datetime64[ns]


In [16]:
tot_count = df.groupby(['Primary Type','Temp_Farenheit']).count()
crime_categories.rename(columns={'crime primary type':'Primary Type'},inplace=True)
combined_df = df.merge(crime_categories, how='inner',on='Primary Type')
combined_df

,Unnamed: 0,Case Number,Primary Type,datetime,Temp_Farenheit,bucket
0,0,JA119772,THEFT,2016-01-12 23:00:00,39.002000,Property
1,15,JA138825,THEFT,2016-01-12 23:00:00,39.002000,Property
2,18,JA100658,THEFT,2016-01-12 23:00:00,39.002000,Property
3,19,JA100309,THEFT,2016-01-12 23:00:00,39.002000,Property
4,33,JA101421,THEFT,2016-01-12 23:00:00,39.002000,Property
5,34,JA109885,THEFT,2016-01-12 23:00:00,39.002000,Property
6,37,JA101089,THEFT,2016-01-12 23:00:00,39.002000,Property
7,39,HZ571828,THEFT,2016-01-12 23:00:00,39.002000,Property
8,40,JA100129,THEFT,2016-01-12 23:00:00,39.002000,Property
9,41,JA104488,THEFT,2016-01-12 23:00:00,39.002000,Property
